<a href="https://colab.research.google.com/github/BTT-Cadence-Design-Systems-2A/AI-Studio-Project/blob/Bert-base-cased-model/Cadence_2A_Bert_base_cased_model(ABSA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Install libraries**

In [ ]:
!pip install -U datasets huggingface_hub

**Imports & config**

In [ ]:
import json
import fsspec
from itertools import islice
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download("punkt")
nltk.download("wordnet")
nltk.download("omw-1.4")
nltk.download("punkt_tab")

REPO = "McAuley-Lab/Amazon-Reviews-2023"


CATEGORIES = ["Software", "Video_Games", "All_Beauty"]
ALL_CATEGORIES = ["All_Beauty", "Amazon_Fashion", "Appliances", "Arts_Crafts_and_Sewing", "Automotive", "Baby_Products", "Beauty_and_Personal_Care", "Books",
              "CDs_and_Vinyl", "Cell_Phones_and_Accessories", "Clothing_Shoes_and_Jewelry", "Digital_Music", "Electronics", "Gift_Cards", "Grocery_and_Gourmet_Food",
              "Handmade_Products", "Health_and_Household", "Health_and_Personal_Care", "Home_and_Kitchen", "Industrial_and_Scientific",
              "Kindle_Store", "Magazine_Subscriptions", "Movies_and_TV", "Musical_Instruments", "Office_Products", "Patio_Lawn_and_Garden", "Pet_Supplies",
              "Software", "Sports_and_Outdoors", "Subscription_Boxes", "Tools_and_Home_Improvement", "Toys_and_Games", "Video_Games",
              "Unknown"]


N_PER_CAT = 10_000
N_META    = 60_000

pd.set_option("display.max_colwidth", 200)

**Load & sample each category (streaming) and concatenate**

In [ ]:
def stream_jsonl(url: str, limit: int | None = None):
    """
    Stream a JSONL file line-by-line from Hugging Face
    Normalizes mixed-type fields like 'price'
    """
    with fsspec.open(url, "rt") as f:
        for idx, line in enumerate(f):
            if limit is not None and idx >= limit:
                break
            obj = json.loads(line)


            if "price" in obj and obj["price"] is not None:
                obj["price"] = str(obj["price"])

            return_obj = obj
            yield return_obj


def ensure_asin(df: pd.DataFrame) -> pd.DataFrame:
    """
    Ensure there is an 'asin' column
    """
    for cand in ["asin", "parent_asin", "product_id", "item_id", "Parent_ASIN", "ParentAsin"]:
        if cand in df.columns:
            if "asin" not in df.columns:
                df["asin"] = df[cand]
            return df
    if len(df) > 0:
        print("No recognizable ASIN-like key found. Example row:\n", df.head(1).to_dict("records")[0])
    return df


def load_category(category: str, n_reviews: int, n_meta: int):
    """
    Load one category's reviews and meta as DataFrames
    """
    reviews_url = f"hf://datasets/{REPO}/raw/review_categories/{category}.jsonl"
    meta_url    = f"hf://datasets/{REPO}/raw/meta_categories/meta_{category}.jsonl"

    reviews_df = pd.DataFrame(islice(stream_jsonl(reviews_url), n_reviews)).assign(category=category)
    meta_df    = pd.DataFrame(islice(stream_jsonl(meta_url),    n_meta)).assign(category=category)
    return reviews_df, meta_df

**Inspect schemas and key columns**

In [ ]:
all_reviews, all_meta = [], []

for cat in CATEGORIES:
    r_df, m_df = load_category(cat, n_reviews=N_PER_CAT, n_meta=N_META)
    all_reviews.append(r_df)
    all_meta.append(m_df)

reviews_df = pd.concat(all_reviews, ignore_index=True)
meta_df    = pd.concat(all_meta,    ignore_index=True)

reviews_df = ensure_asin(reviews_df)
meta_df    = ensure_asin(meta_df)


if "asin" in reviews_df:
    reviews_df = reviews_df[reviews_df["asin"].notna()]
if "asin" in meta_df:
    meta_df = meta_df[meta_df["asin"].notna()]

print(f"Loaded rows -> reviews: {len(reviews_df):,} | meta: {len(meta_df):,}")
display(reviews_df.head(2))
display(meta_df.head(2))

print(f"Unique products in reviews: {reviews_df['asin'].nunique():,}")
print(f"Unique products in meta: {meta_df['asin'].nunique():,}")


In [ ]:
meta_df.columns

**Helper: ensure_asin + normalize IDs**

In [ ]:
meta_keys = {"asin", "parent_asin", "category", "title"}
meta_keep = ["asin", "parent_asin", "title"] + [c for c in meta_df.columns if c not in meta_keys]


m1 = reviews_df.merge(meta_df[meta_keep], on="asin", how="left", suffixes=("_review", "_meta"))
display(m1)
m1.columns

# m2 = reviews_df.merge(
#     meta_df[meta_keep].rename(columns={"asin": "asin_meta2", "parent_asin": "parent_asin_meta2"}),
#     left_on="parent_asin",
#     right_on="asin_meta2",
#     how="left",
# )


# merged = m1.copy()
# for col in meta_keep:
#     if col in {"asin", "parent_asin"}:
#         continue
#     col_m1 = col
#     col_m2 = col + "_m2"
#     if col in m2.columns:
#         merged[col_m2] = m2[col]
#         merged[col] = merged[col].where(merged[col].notna(), merged[col_m2])
#         merged.drop(columns=[col_m2], inplace=True)


# if "asin_meta2" in m2.columns:
#     merged["asin_meta_fallback"] = m2["asin_meta2"]

# print("Merged shape:", merged.shape)


# meta_signal = [c for c in merged.columns if c.endswith("_meta") or c in ["average_rating", "rating_number", "price", "store", "categories", "details", "title", "images", "videos", "main_category"]]
# coverage = merged[meta_signal].notna().any(axis=1).mean() if meta_signal else 0.0
# print(f"Rows with ANY meta fields: {coverage:.2%}")

# display(merged.head(5))

# **Milestone #1: Sentiment Analysis of a Singular Review**


Goal: Take the reviews dataframe, only maintain the rating, title, category, and text columns, and then train a model that predicts the rating given a review text


In [ ]:
# loading necessary column of review dataset
def load_category_into_review(category: str, n_reviews: int):
    """
    Load one category's reviews as DataFrames
    """
    reviews_url = f"hf://datasets/{REPO}/raw/review_categories/{category}.jsonl"

    data = (
        {k: row.get(k) for k in ["rating", "title", "text", "parent_asin"]}
        for row in islice(stream_jsonl(reviews_url), n_reviews)
    )

    reviews_df = pd.DataFrame(data).assign(category=category)
    return reviews_df

In [ ]:
# def load_meta_data_into_review(category: str, m_reviews:int):
#   '''
#   Load one category's meta data as DataFrames

#   '''
#   meta_url = f"hf://datasets/{REPO}/raw/meta_categories/meta_{category}.jsonl"
#   data = (
#       {k: row.get(k) for k in ["parent_asin", "title"]}
#       for row in islice(stream_jsonl(meta_url), m_reviews)
#   )
#   meta_df = pd.DataFrame(data).assign(category=category)
#   return meta_df


In [ ]:

sentiment_reviews =  []

for cat in ALL_CATEGORIES:
    r_df = load_category_into_review(cat, n_reviews=N_PER_CAT) # review data
    # m_df = load_meta_data_into_review(cat, m_reviews=N_META) # meta data
    sentiment_reviews.append(r_df)
    # print(f"{cat} is appended")

reviews_df_milestone1 = pd.concat(sentiment_reviews, ignore_index=True)


print(f"Loaded rows -> reviews: {len(reviews_df_milestone1):,}")
display(reviews_df_milestone1.head(2))

In [ ]:
reviews_df_milestone1.info()
reviews_df_milestone1['rating'].value_counts()

## Milestone #1: Data Cleaning

In [ ]:
reviews_df_milestone1.isna().sum()

### Text Normalization (removing punctuation)

In [ ]:
import string


def remove_punctuation(text: str) -> str:
    """
    Function removes all punctuation from a string
    """
    if not isinstance(text, str):
        return ""
    return text.translate(str.maketrans("", "", string.punctuation))

In [ ]:
"""
   Creates clean_review and clean_title and clean_review. These two columns will be used during model training.
"""
reviews_df_milestone1['clean_review'] = (
    reviews_df_milestone1['text']
    .str.lower()
    .apply(remove_punctuation)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

reviews_df_milestone1['clean_title'] = (
    reviews_df_milestone1['title']
    .str.lower()
    .apply(remove_punctuation)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

### Lemmitization of Reviews

In [ ]:
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    tokens = word_tokenize(text)
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    return " ".join(lemmas)

In [ ]:
reviews_df_milestone1['lemmatized_review'] = reviews_df_milestone1['clean_review'].apply(lemmatize_text)
reviews_df_milestone1['lemmatized_title'] = reviews_df_milestone1['clean_title'].apply(lemmatize_text)

### Creating Sentiment Labels


In [ ]:
def create_sentiment_label(rating: int) -> str:
  if rating >= 4:
    return 'positive'
  elif rating <= 2:
    return 'negative'
  else:
    return 'neutral'

In [ ]:
reviews_df_milestone1['sentiment_labels'] = (
    reviews_df_milestone1['rating']
    .apply(create_sentiment_label)
)

In [ ]:
reviews_df_milestone1.head()

### Tokenization of Reviews


In [ ]:
# documents = reviews_df_milestone1['clean_review'].tolist()

In [ ]:
# vectorizer = TfidfVectorizer(
#     stop_words="english",   # remove english stopwords like this, a, the, etc
#     # max_features=5000,      # keep top 5000 words (tune this)
# )
# X = vectorizer.fit_transform(documents)

In [ ]:
# print(f"Vocabulary size: {len(vectorizer.vocabulary_)}")

# df_tfidf = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
# df_tfidf.head()

In [ ]:
from nltk.tokenize import word_tokenize
reviews_df_milestone1['tokenized_review'] = reviews_df_milestone1['clean_review'].apply(word_tokenize)

In [ ]:
reviews_df_milestone1.head(5)

### Creating another column for sentiment label classes
For this label, we will set 0 - negative, 1 - neutral, 2 - positive.We need this to calculate loss between model output and the ground truth sentiment labels.

In [ ]:
def create_sentiment_label_classes(rating: int) -> str:
  if rating >= 4:
    return 2
  elif rating <= 2:
    return 0
  else:
    return 1

In [ ]:
reviews_df_milestone1['sentiment_label_classes'] = (
    reviews_df_milestone1['rating']
    .apply(create_sentiment_label_classes)
)

In [ ]:
reviews_df_milestone1.head(10)

# Bert-base Cased Model for sentiment analysis

### Install libraries

In [ ]:
!pip install transformers
!pip install torch
!pip install datasets

### Convert "clean review" column to Dataset

In [ ]:
from datasets import Dataset
clean_reviews_dataset = Dataset.from_pandas(reviews_df_milestone1[['clean_review', 'parent_asin','sentiment_label_classes']]).rename_column("sentiment_label_classes", "ground_truth_labels")

In [ ]:
print(clean_reviews_dataset)

### Load model and model's tokenizer to convert cleaned review text to number embeddings

In [ ]:
# Load model directly
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# define the tokenization function
def tokenize_text(examples):
  return tokenizer(examples['clean_review'], padding=True, truncation=True, max_length=128)

# apply tokenization func to the clean review text
tokenized_clean_reviews = clean_reviews_dataset.map(tokenize_text, batched=True)

In [ ]:
print(tokenized_clean_reviews)

### Split the tokenized text into training, validation, and test set
We will use 70% training set, 15% validation set and 15% test set

In [ ]:
train_dataset, valid_test_dataset = tokenized_clean_reviews.train_test_split(test_size=0.3, seed=42).values()
valid_dataset, test_dataset = valid_test_dataset.train_test_split(test_size=0.5, seed=42).values()

In [ ]:
print(train_dataset)
print(f"train dataset shape: {train_dataset.shape}")
print(valid_dataset)
print(f"valid dataset shape: {valid_dataset.shape}")
print(test_dataset)
print(f"test dataset shape: {test_dataset.shape}")

### Create data loader to manage batches of data during training
Dataloader is used to organize data for model training by providing efficient ways to batch, shuffle, and transform data.


In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8)
valid_dataloader = DataLoader(valid_dataset, batch_size=8)
test_dataloader = DataLoader(test_dataset, batch_size=8)

### Setting up the model and config for fine-tuning the model

AdamW is for adjusting learning rate during training.




In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from torch.optim import AdamW

# load the pre-trained bert model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# define training argument
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Bert-base Multilingual Uncase Model checkpoint', # ouput directory for model checkpoint
    eval_strategy='epoch', # evaluation strategy
    num_train_epochs=3, # num of training epochs
    learning_rate=2e-5, # learning rate
    per_device_train_batch_size=8, # batch size for training
    per_device_eval_batch_size=8, # batch size for evaluation
    weight_decay=0.01 # weight decay

)

In [ ]:
# define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset
)

In [ ]:
trainer.train() # Train the model

### Evaluating the model

In [ ]:
!pip install evaluate

In [ ]:
res = trainer.predict(test_dataset)
logits = res.predictions # pred scores for each class
labels = res.label_ids # label for each class
pred = logits.argmax(axis=-1)   # pick the class with highest score

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

acc = accuracy_score(labels, pred)
prec, rec, f1, _ = precision_recall_fscore_support(labels, pred, average="weighted")

print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1 score:", f1)


### Inference of the fine-tuned model

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer, Trainer

'''
SAVING THE TOKENIZER IN THE SAVED CHECKPOINT
finetuned_model_path = "/content/drive/MyDrive/Bert-base Multilingual Uncase Model checkpoint/checkpoint-89250"

## Weight and parameters of the pre-trained model's tokenizer never changed
# reload original pretrained tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")

# save tokenizer into your finetuned model folder
tokenizer.save_pretrained(finetuned_model_path)
'''

In [ ]:
finetuned_model_path = "/content/drive/MyDrive/checkpoint-89250" # change file paths based on where you store the model checkpoint

# load the finetuned model
model = BertForSequenceClassification.from_pretrained(finetuned_model_path)
# load the tokenizer in the finetuned model
tokenizer = BertTokenizer.from_pretrained(finetuned_model_path)

trainer = Trainer(model=model)

# Running prediction
res = trainer.predict(test_dataset)
logits = res.predictions # pred scores for each class
labels = res.label_ids # label for each class
pred = logits.argmax(axis=-1)

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# print(pred)
acc = accuracy_score(labels, pred)
prec, rec, f1, _ = precision_recall_fscore_support(labels, pred, average="weighted")

print("Accuracy: ", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1 score:", f1)

### Comparison of Original Bert vs. Finetuned Bert model


In [ ]:
# running the original bert-base cased model on the test dataset
from transformers import BertForSequenceClassification, Trainer
num_labels = 3  # adjust to your dataset

orig_model = BertForSequenceClassification.from_pretrained(
    "bert-base-cased",
    num_labels=num_labels
)
orig_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

orig_trainer = Trainer(model=orig_model)
res = orig_trainer.predict(test_dataset)


In [ ]:
logits = res.predictions # pred scores for each class
labels = res.label_ids # label for each class
orig_model_pred = logits.argmax(axis=-1)

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# print(pred)
orig_acc = accuracy_score(labels, orig_model_pred)
orig_prec, orig_rec, orig_f1, _ = precision_recall_fscore_support(labels, orig_model_pred, average="weighted")

print("Accuracy: ", orig_acc)
print("Precision:", orig_prec)
print("Recall:", orig_rec)
print("F1 score:", orig_f1)

### Visualization of the accuracy of Original vs. Finetuned model

In [ ]:
import matplotlib.pyplot as plt

metrics = ["Accuracy", "F1 Score"]
orig_values = [orig_acc, orig_f1]
finetuned_values = [acc, f1]

x = range(len(metrics))
width = 0.35

plt.bar([i - width/2 for i in x], orig_values, width=width, label="Original (Pretrained)")
plt.bar([i + width/2 for i in x], finetuned_values, width=width, label="Fine-tuned")

plt.xticks(x, metrics)
plt.ylabel("Score")
plt.ylim(0, 1)
plt.legend()
plt.title("Original Bert-base cased vs Fine-tuned Performance")
plt.show()

# Aspect Based Sentiment Analysis
Aspect Based Sentiment Analysis (ABSA) is used to extract features that got good or bad reviews from customers to get insights about the good features and bad features of specific products.


In [ ]:
!pip install pyabsa

### Defining the Aspect Extractor

In [ ]:
from pyabsa import ATEPCCheckpointManager

extractor = ATEPCCheckpointManager.get_aspect_extractor(
    checkpoint='multilingual',
    auto_device=True
)


### Extracting Positive, Negative, and Neutral Aspects (Features) in the Reivew Products
The features are extracted along with their corresponding sentiments in the product reveiws.

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
import torch, torch.nn.functional as F
import pandas as pd

finetuned_model_path = "/content/drive/MyDrive/checkpoint-89250" # change file paths based on where you

# load the finetuned model
model = BertForSequenceClassification.from_pretrained(finetuned_model_path)
# load the tokenizer in the finetuned model
tokenizer = BertTokenizer.from_pretrained(finetuned_model_path)

args = TrainingArguments(
    output_dir="tmp",
    per_device_eval_batch_size=8

)
trainer = Trainer(model=model, args=args, tokenizer=tokenizer)

# Running prediction
res = trainer.predict(test_dataset)
logits = res.predictions # pred scores for each class by the finetuned model
pred = logits.argmax(axis=-1) # find the index of largest class and assign that class to review

# probs and label strings
probs = F.softmax(torch.tensor(logits), dim=-1)

predicted_labels = [p for p in pred]


### Restructuring the dataset
The test dataset is restructured into 'review', 'label', 'predicted label'. The positive and aspect features will be added to each review in the dataset.

In [ ]:
# make a dataset ['clean_review', 'ground truth labels']
cols_keep = ["clean_review", "ground truth labels"]
df_test = test_dataset.remove_columns(
    [col for col in test_dataset.column_names if col not in cols_keep]
)
df_test = df_test.to_pandas()

In [ ]:
# adding predicted labels into the dataset
df_test['predicted_labels'] = predicted_labels
display(df_test.head(5))

In [ ]:
# Converting the numerical labels to string labels
idx2name = {0: "negative", 1: "neutral", 2:"positive"}
df_test['predicted_labels'] = [idx2name[label] for label in df_test['predicted_labels']]
df_test['ground truth labels'] = [idx2name[label] for label in df_test['ground truth labels']]

In [ ]:
print(df_test.columns)
display(df_test.head(5))

### Adding Positive, Negative, and Neutral features of the products in the dataset

In [ ]:
reviews = df_test["clean_review"].tolist()
results = extractor.extract_aspect(inference_source=reviews, pred_sentiment=True)

n = len(reviews)
good_aspects = [[] for _ in range(n)]
bad_aspects = [[] for _ in range(n)]
neutral_aspects = [[] for _ in range(n)]


In [ ]:
for i, r in enumerate(results):
  aspects = r.get("aspect") or r.get("aspects") or []
  sents = r.get("sentiment") or r.get("sentiments") or []

  for a, s in zip(aspects, sents):
    label = (str(s) or "").strip().lower()

    if "neg" in label:
      bad_aspects[i].append(a)
    elif "pos" in label:
      good_aspects[i].append(a)
    elif "neu" in label or label == 'neutral':
      neutral_aspects[i].append(a)
    else:
      neutral_aspects[i].append(a)

In [ ]:
import numpy as np

def empty_to_nan(lst):
  return np.nan if not lst else lst

df_test["Positive Review Product Features"] = [empty_to_nan(x) for x in good_aspects]
df_test["Negative Review Product Features"] = [empty_to_nan(x) for x in bad_aspects]
df_test["Neutral Review Product Features"] = [empty_to_nan(x) for x in neutral_aspects]

In [ ]:
df_test = df_test.drop("Bad Review Product Features", axis=1)
display(df_test.head(5))

### Saving the dataset as csv file

The reviews are saved in a csv file along with their ground truth labels (positive, negative, neutral based on ratings), predicted labels from Bert based cased model, extracted positive, negative, and neutral features.



In [ ]:
output_csv = "/content/drive/MyDrive/Final_test_dataset.csv"
df_test.to_csv(output_csv, index=False)